In [1]:
import pandas as pd

In [2]:
# Read the CSV dataset which it was originally an html file exported from ERP system
df = pd.read_csv('emp_fingerprints.csv', encoding='latin-1')
# Show this original dataframe
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Employees attendance,Employees attendance.1,Employees attendance.2,Employees attendance.3,Unnamed: 7,Unnamed: 8,Unnamed: 9,Global Trading Co.,Global Trading Co..1
0,NaN,NaN,All Branches,26/09/2023,26/09/2023,Dates from:,Dates from:,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,26/10/2023,26/10/2023,To:,To:,NaN,NaN,NaN,NaN,NaN
2,Employee ID,Employee Name,Employee Name,Employee Name,date,date,time,time,In / Out,In / Out,NaN,NaN
3,2074,Mohamed Adam,Mohamed Adam,Mohamed Adam,26/09/2023,26/09/2023,03:03:43,03:03:43,OUT,OUT,NaN,NaN
4,2074,Mohamed Adam,Mohamed Adam,Mohamed Adam,26/09/2023,26/09/2023,16:56:08,16:56:08,OUT,OUT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
145,2074,Mohamed Adam,Mohamed Adam,Mohamed Adam,25/10/2023,25/10/2023,03:13:44,03:13:44,OUT,OUT,NaN,NaN
146,2074,Mohamed Adam,Mohamed Adam,Mohamed Adam,25/10/2023,25/10/2023,03:13:50,03:13:50,OUT,OUT,NaN,NaN
147,2074,Mohamed Adam,Mohamed Adam,Mohamed Adam,25/10/2023,25/10/2023,03:13:52,03:13:52,OUT,OUT,NaN,NaN
148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Get and rename the required columns and rows
cleaned_df = df.iloc[3:-2,[0,1,4,6,8]]
cleaned_df.columns = ['emp_id', 'emp_name', 'date', 'time', 'mode']
# convert [date] and [time] from object to datetime and timedelta
cleaned_df['date'] = pd.to_datetime(cleaned_df['date'], errors='coerce', dayfirst= True)
cleaned_df['time'] = pd.to_timedelta(cleaned_df['time'], errors='coerce')
# Create a mask "mask" to remove any row with [time] = 0:00:00, 00:00:00, or 23:59:59 because we don't need them for our analysis
mask = ~ cleaned_df['time'].isin(['0:00:00','00:00:00','23:59:59'])
cleaned_df = cleaned_df[mask].reset_index(drop=True)
# Create a new column [date_time] that combines [date] and [time]
cleaned_df['date_time'] = cleaned_df['date'] + cleaned_df['time']
# Check our columns data types are correct and then show the dataframe
cleaned_df.info()
cleaned_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   emp_id     99 non-null     object         
 1   emp_name   99 non-null     object         
 2   date       99 non-null     datetime64[ns] 
 3   time       99 non-null     timedelta64[ns]
 4   mode       99 non-null     object         
 5   date_time  99 non-null     datetime64[ns] 
dtypes: datetime64[ns](2), object(3), timedelta64[ns](1)
memory usage: 4.8+ KB


,emp_id,emp_name,date,time,mode,date_time
0,2074,Mohamed Adam,2023-09-26,0 days 03:03:43,OUT,2023-09-26 03:03:43
1,2074,Mohamed Adam,2023-09-26,0 days 16:56:08,OUT,2023-09-26 16:56:08
2,2074,Mohamed Adam,2023-09-26,0 days 16:56:09,IN,2023-09-26 16:56:09
3,2074,Mohamed Adam,2023-09-27,0 days 03:10:59,IN,2023-09-27 03:10:59
4,2074,Mohamed Adam,2023-09-27,0 days 03:11:01,OUT,2023-09-27 03:11:01
...,...,...,...,...,...,...
94,2074,Mohamed Adam,2023-10-24,0 days 17:13:06,IN,2023-10-24 17:13:06
95,2074,Mohamed Adam,2023-10-25,0 days 03:13:43,OUT,2023-10-25 03:13:43
96,2074,Mohamed Adam,2023-10-25,0 days 03:13:44,OUT,2023-10-25 03:13:44
97,2074,Mohamed Adam,2023-10-25,0 days 03:13:50,OUT,2023-10-25 03:13:50


In [4]:
# Create new masks "in_duplicates" and "out_duplicates" to ensure that the mode in [filter] is "in" or "out" correctly
# We use this step as [mode] column isn't trustable because employees often don't give enough attention to the device mode before checking in or out
in_duplicates = (cleaned_df['time'] >= pd.Timedelta('0 days 15:00:00')) & (cleaned_df['time'] <= pd.Timedelta('0 days 20:00:00'))
cleaned_df.loc[in_duplicates, 'filter'] = 'in'
cleaned_df_in = cleaned_df[cleaned_df['filter']=='in']
# The same steps but for "OUT" mode
out_duplicates = (cleaned_df['time'] >= pd.Timedelta('0 days 23:00:00')) | (cleaned_df['time'] <= pd.Timedelta('0 days 04:00:00'))
cleaned_df.loc[out_duplicates, 'filter'] = 'out'
cleaned_df_out = cleaned_df[cleaned_df['filter']=='out']

In [5]:
# Use "mask_keep_in" to keep only the first occurance of "in" for each unique date
# and "mask_keep_out" to keep only the last occurance of "out" for each unique date
mask_keep_in = ~ cleaned_df_in[['date','filter']].duplicated()
mask_keep_out = ~ cleaned_df_out[['date','filter']].duplicated(keep='last')
# Concatenate both masks
appended_masks = pd.concat([mask_keep_in, mask_keep_out], axis=0).sort_index()
# Use "appended_masks" mask to ensure that our cleaned_df is not duplicated anymore
no_duplicates_df = cleaned_df.loc[appended_masks][['emp_id', 'emp_name', 'date', 'date_time', 'filter']]
display(no_duplicates_df.head())
no_duplicates_df.tail()

,emp_id,emp_name,date,date_time,filter
0,2074,Mohamed Adam,2023-09-26,2023-09-26 03:03:43,out
1,2074,Mohamed Adam,2023-09-26,2023-09-26 16:56:08,in
4,2074,Mohamed Adam,2023-09-27,2023-09-27 03:11:01,out
5,2074,Mohamed Adam,2023-09-28,2023-09-28 17:05:29,in
8,2074,Mohamed Adam,2023-09-29,2023-09-29 03:14:09,out


,emp_id,emp_name,date,date_time,filter
88,2074,Mohamed Adam,2023-10-23,2023-10-23 03:14:28,out
89,2074,Mohamed Adam,2023-10-23,2023-10-23 17:10:44,in
92,2074,Mohamed Adam,2023-10-24,2023-10-24 03:04:11,out
93,2074,Mohamed Adam,2023-10-24,2023-10-24 17:13:05,in
98,2074,Mohamed Adam,2023-10-25,2023-10-25 03:13:52,out


In [6]:
# Transform our no_duplicates_df to have [check_in] and [check_out]
no_duplicates_df.loc[no_duplicates_df['filter']=='in', 'check_in'] = no_duplicates_df['date_time']
no_duplicates_df.loc[no_duplicates_df['filter']=='out', 'check_out'] = no_duplicates_df['date_time']
no_duplicates_df.head()

,emp_id,emp_name,date,date_time,filter,check_in,check_out
0,2074,Mohamed Adam,2023-09-26,2023-09-26 03:03:43,out,NaT,2023-09-26 03:03:43
1,2074,Mohamed Adam,2023-09-26,2023-09-26 16:56:08,in,2023-09-26 16:56:08,NaT
4,2074,Mohamed Adam,2023-09-27,2023-09-27 03:11:01,out,NaT,2023-09-27 03:11:01
5,2074,Mohamed Adam,2023-09-28,2023-09-28 17:05:29,in,2023-09-28 17:05:29,NaT
8,2074,Mohamed Adam,2023-09-29,2023-09-29 03:14:09,out,NaT,2023-09-29 03:14:09


In [7]:
# Shift the check_out column to the next row
no_duplicates_df['check_out'] = (list(no_duplicates_df.loc[1:,'check_out']) + ['fill'])
# Remove NaT rows
no_duplicates_df = no_duplicates_df[no_duplicates_df['check_in'].notna()]
# Convert [check_out] to datetime again because it was converted to object in the shifting step
no_duplicates_df['check_out'] = pd.to_datetime(no_duplicates_df['check_out'])
no_duplicates_df.head()

,emp_id,emp_name,date,date_time,filter,check_in,check_out
1,2074,Mohamed Adam,2023-09-26,2023-09-26 16:56:08,in,2023-09-26 16:56:08,2023-09-27 03:11:01
5,2074,Mohamed Adam,2023-09-28,2023-09-28 17:05:29,in,2023-09-28 17:05:29,2023-09-29 03:14:09
9,2074,Mohamed Adam,2023-09-29,2023-09-29 17:12:07,in,2023-09-29 17:12:07,2023-09-30 03:22:29
13,2074,Mohamed Adam,2023-09-30,2023-09-30 17:00:00,in,2023-09-30 17:00:00,2023-10-01 03:03:45
17,2074,Mohamed Adam,2023-10-01,2023-10-01 17:08:52,in,2023-10-01 17:08:52,2023-10-02 03:06:38


In [8]:
# Calculate hours
no_duplicates_df['hours'] = no_duplicates_df['check_out'] - no_duplicates_df['check_in']
# Convert [hours] from '0 days 01:30:00' format into '1.50' format
no_duplicates_df['hours'] = no_duplicates_df['hours'].apply(lambda x: pd.Timedelta(x).total_seconds() / 3600)
# Drop unnecessary columns
no_duplicates_df.drop(['date_time', 'filter'], axis=1, inplace= True)
no_duplicates_df.info()
display(no_duplicates_df.head())
no_duplicates_df.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 93
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   emp_id     23 non-null     object        
 1   emp_name   23 non-null     object        
 2   date       23 non-null     datetime64[ns]
 3   check_in   23 non-null     datetime64[ns]
 4   check_out  23 non-null     datetime64[ns]
 5   hours      23 non-null     float64       
dtypes: datetime64[ns](3), float64(1), object(2)
memory usage: 1.3+ KB


,emp_id,emp_name,date,check_in,check_out,hours
1,2074,Mohamed Adam,2023-09-26,2023-09-26 16:56:08,2023-09-27 03:11:01,10.248056
5,2074,Mohamed Adam,2023-09-28,2023-09-28 17:05:29,2023-09-29 03:14:09,10.144444
9,2074,Mohamed Adam,2023-09-29,2023-09-29 17:12:07,2023-09-30 03:22:29,10.172778
13,2074,Mohamed Adam,2023-09-30,2023-09-30 17:00:00,2023-10-01 03:03:45,10.062500
17,2074,Mohamed Adam,2023-10-01,2023-10-01 17:08:52,2023-10-02 03:06:38,9.962778


,emp_id,emp_name,date,check_in,check_out,hours
77,2074,Mohamed Adam,2023-10-20,2023-10-20 17:01:43,2023-10-21 03:17:43,10.266667
81,2074,Mohamed Adam,2023-10-21,2023-10-21 17:04:29,2023-10-22 03:21:34,10.284722
85,2074,Mohamed Adam,2023-10-22,2023-10-22 17:17:40,2023-10-23 03:14:28,9.946667
89,2074,Mohamed Adam,2023-10-23,2023-10-23 17:10:44,2023-10-24 03:04:11,9.890833
93,2074,Mohamed Adam,2023-10-24,2023-10-24 17:13:05,2023-10-25 03:13:52,10.013056


In [9]:
# Create pivot table to know the total hours for each employee (it will be useful wen you have more than one empoyee)
no_duplicates_df.pivot_table(index = ['emp_id', 'emp_name'], values= 'hours', aggfunc='sum')

,,hours
emp_id,emp_name,
2074,Mohamed Adam,231.686389


In [10]:
no_duplicates_df

,emp_id,emp_name,date,check_in,check_out,hours
1,2074,Mohamed Adam,2023-09-26,2023-09-26 16:56:08,2023-09-27 03:11:01,10.248056
5,2074,Mohamed Adam,2023-09-28,2023-09-28 17:05:29,2023-09-29 03:14:09,10.144444
9,2074,Mohamed Adam,2023-09-29,2023-09-29 17:12:07,2023-09-30 03:22:29,10.172778
13,2074,Mohamed Adam,2023-09-30,2023-09-30 17:00:00,2023-10-01 03:03:45,10.062500
17,2074,Mohamed Adam,2023-10-01,2023-10-01 17:08:52,2023-10-02 03:06:38,9.962778
21,2074,Mohamed Adam,2023-10-02,2023-10-02 17:19:47,2023-10-03 03:08:44,9.815833
26,2074,Mohamed Adam,2023-10-03,2023-10-03 17:07:39,2023-10-04 03:13:05,10.090556
30,2074,Mohamed Adam,2023-10-05,2023-10-05 17:00:00,2023-10-06 03:16:07,10.268611
34,2074,Mohamed Adam,2023-10-06,2023-10-06 17:11:16,2023-10-07 03:21:07,10.164167
38,2074,Mohamed Adam,2023-10-07,2023-10-07 17:11:43,2023-10-08 03:17:35,10.097778


In [11]:
start_date = cleaned_df['date'].min()
end_date = cleaned_df['date'].max()
date_range = pd.date_range(start_date, end_date)
missing_dates = list(date_range[~date_range.isin(no_duplicates_df['date'])])
missing_dates2 = [i.date().strftime('%Y-%m-%d') for i in missing_dates]
missing_dates_df = pd.DataFrame({'date': missing_dates2,
                                 'emp_id': no_duplicates_df['emp_id'][9],
                                 'emp_name': no_duplicates_df['emp_name'][9]})
missing_dates_df

,date,emp_id,emp_name
0,2023-09-27,2074,Mohamed Adam
1,2023-10-04,2074,Mohamed Adam
2,2023-10-11,2074,Mohamed Adam
3,2023-10-13,2074,Mohamed Adam
4,2023-10-14,2074,Mohamed Adam
5,2023-10-18,2074,Mohamed Adam
6,2023-10-25,2074,Mohamed Adam
